In [2]:
# Single Dispatch Generic Functions
# Sidenote:: Start building a use case for the knowledge you get

In [21]:
from html import escape

In [22]:
def html_escape(arg):
    return escape(str(arg))
def html_int(a):
    return '{0}<i>{1}<i>'.format(a, str(hex(a)))
def html_real(a):
    return '{0:2f}'.format(round(a, 5))
def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')
def html_list(l):
    items = ('<li>{0}</li>'.format(html_escape(item)) for item in l) 
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'
def html_dict(d):
    items = ('<li>{0}={1}</li>'.format(k,v) for (k,v) in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [23]:
print(html_str("""
This is a great line endhere
that shows html str
with colon: number 20 and gt
"""))

<br/>
This is a great line endhere<br/>
that shows html str<br/>
with colon: number 20 and gt<br/>



In [24]:
print(html_int(8))

8<i>0x8<i>


In [25]:
from decimal import Decimal # real numbers
print(html_real(Decimal('3.2')))


3.20000


In [26]:
def single_dispatch(fn):
    from html import escape
    registry = {}
    registry[object] = fn
    registry[int] = lambda a: '{0}<i>{1}<i>'.format(a, str(hex(a)))
    registry[str] = lambda a: escape(str(a)).replace('\n', '<br/>\n')
    def inner(arg):  
        return registry.get(type(arg), object)(arg)
    return inner
        

In [27]:
@single_dispatch
def htmlize(a):
    from html import escape
    return escape(str(a))

In [28]:
htmlize('a < b')

'a &lt; b'

In [29]:
htmlize(100)

'100<i>0x64<i>'

In [30]:
htmlize("kct 199 [3]")

'kct 199 [3]'

In [48]:
def single_dispatch(fn):
    from html import escape
    registry = {}
    registry[object] = fn
    def decorated(arg):  
        return registry.get(type(arg), registry[object])(arg)
    def register(type_): # decorator factory
        def inner(fn): # only one level of nested closure
            registry[type_] = fn
            return fn
        return inner
    decorated.register = register
    decorated.registry = registry
    return decorated
        

In [35]:
@single_dispatch
def htmlize(a):
    return(escape(str(a)))

In [36]:
htmlize("3 < 2")

'3 &lt; 2'

In [37]:
htmlize.register

<function __main__.single_dispatch.<locals>.register>

In [40]:
@htmlize.register(int)
def html_int(a):
     return '{0}<i>{1}<i>'.format(a, str(hex(a)))

In [41]:
htmlize(100)

'100<i>0x64<i>'

In [42]:
htmlize([3])

'[3]'

In [45]:
@htmlize.register(list)
@htmlize.register(tuple)
def html_sequence(l): # html_list will be injected into the registry dictionary
    items = ('<li>{0}</li>'.format(html_escape(item)) for item in l) 
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [46]:
htmlize([3, 5])

'<ul>\n<li>3</li>\n<li>5</li>\n</ul>'

In [51]:
@single_dispatch
def get_attribute():
    pass

In [52]:
get_attribute.registry

{object: <function __main__.get_attribute>}

In [12]:
def single_dispatch(fn):
    from html import escape
    registry = {}
    registry[object] = fn
    def decorated(arg):  
        return registry.get(type(arg), registry[object])(arg)
    def register(type_): # decorator factory
        def inner(fn): # only one level of nested closure
            registry[type_] = fn
            return fn
        return inner
    def dispatch(type_):
        return registry.get(type_, registry[object])
    decorated.register = register
    decorated.dispatch = dispatch
#     decorated.registry = registry : Not a good idea to allow people to directly access registry
    
    
    return decorated
        

In [13]:
@single_dispatch
def convert_to_html(data):
    return "Failed to convert to html"

In [69]:
convert_to_html(30)

'Failed to convert to html'

In [70]:
@convert_to_html.register(int)
def html_int(a):
    return '{0}<i>{1}<i>'.format(a, str(hex(a))) 

In [71]:
convert_to_html(30)

'30<i>0x1e<i>'

In [72]:
convert_to_html.registry

AttributeError: 'function' object has no attribute 'registry'

In [73]:
@convert_to_html.register(list)
@convert_to_html.register(tuple)
def html_seq(a):
    items = ('<li>{0}</li>'.format(html_escape(item)) for item in l) 
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [75]:
# convert_to_html.registry

In [78]:
convert_to_html.dispatch(int)

'30<i>0x1e<i>'

In [79]:
convert_to_html.dispatch(int)(40)

'40<i>0x28<i>'

In [81]:
@convert_to_html.register(str)
def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

In [82]:
convert_to_html(False)

'Failed to convert to html'

In [14]:
from numbers import Integral
@convert_to_html.register(Integral)
def html_int(a):
    return '{0}<i>{1}<i>'.format(a, str(hex(a))) 

In [3]:
class Person:
    pass

In [4]:
class Student(Person): # student inherits from Person
    pass

In [5]:
p = Student()

In [6]:
type(p)

__main__.Student

In [7]:
isinstance(p, Student)

True

In [8]:
isinstance(p, Person)

True

In [15]:
isinstance(3, Integral)

True

In [16]:
convert_to_html(3)

'Failed to convert to html'

In [18]:
isinstance(True, Integral)

True

In [21]:
from collections.abc import Sequence

In [22]:
isinstance([3, 5], Sequence)

True

In [19]:
convert_to_html(True) # cant convert cuz True is not of TYPE integral

'Failed to convert to html'

In [23]:
# we must find a way to register using isinstance, not type

In [25]:
# Python is known as a batteries included language which means that it actually includes
# a decorator that is in the standard library that does exactly what we have done

In [27]:
from functools import singledispatch
from numbers import Integral
from collections.abc import Sequence

In [28]:
@singledispatch # has a registry attribute
def htmlize(a): # default function that is used for unregistered argument type
    return escape(str(a)) 

In [29]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize>})

In [30]:
htmlize.dispatch(str)

<function __main__.htmlize>

In [31]:
@htmlize.register(Integral)
def htmlize_integral(a):
    return '{0}<i>{1}<i>'.format(a, str(hex(a))) 

In [32]:
htmlize.dispatch(int), htmlize.dispatch(bool)

(<function __main__.htmlize_integral>, <function __main__.htmlize_integral>)

In [33]:
# uses htmlize_integral to handle ints, and bools cuz thy are instances of integral

In [34]:
htmlize(False)

'False<i>0x0<i>'

In [39]:
@htmlize.register(Sequence)
def htmlize_sequence(a):
    items = ('<li>{0}</li>'.format(htmlize(item)) for item in a) 
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [42]:
print(htmlize([2, 3]), htmlize((1, 2)))

<ul>
<li>2<i>0x2<i></li>
<li>3<i>0x3<i></li>
</ul> <ul>
<li>1<i>0x1<i></li>
<li>2<i>0x2<i></li>
</ul>


In [44]:
# htmlize("Pyth") Will return error due to infinite recursion

In [49]:
@htmlize.register(str)
def html_str(s):
    import html
    return html.escape(s).replace('\n', '<br/>\n')

In [50]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize>,
              numbers.Integral: <function __main__.htmlize_integral>,
              collections.abc.Sequence: <function __main__.htmlize_sequence>,
              str: <function __main__.html_str>})

In [53]:
htmlize("proxy"), htmlize('3 >= 5')

('proxy', '3 &gt;= 5')

In [54]:
@htmlize.register(tuple)
def html_tuple(t):
    import html
    items = (htmlize(item) for item )) for item in a)
----> 4     return '<ul>\n' + '\n'.join(items) + '\n</ul>'

<ipython-input-37-54649eaf94bf> in <genexpr>(.0)
      1 @htmlize.register(Sequence)
      2 def htmlize_sequence(a):in t)
    return "<p>{0}<p>".format(', '.join(items))

In [56]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize>,
              numbers.Integral: <function __main__.htmlize_integral>,
              collections.abc.Sequence: <function __main__.htmlize_sequence>,
              str: <function __main__.html_str>,
              tuple: <function __main__.html_tuple>})

In [57]:
print(htmlize((1, 2, 3))) # used html_tuple instead of html_sequence

<p>1<i>0x1<i>, 2<i>0x2<i>, 3<i>0x3<i><p>


In [58]:
8_000

8000

In [59]:
9_7

97

In [61]:
9_0.7_9

90.79

In [62]:
ignored =  43


In [63]:
ignored


43